In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.model_selection import train_test_split

In [88]:
class TwoPointers():
    def __init__(self, col, sep):
        self.col = col
        self.sep = sep
        self.right = None
        self.left = None
        self.left_prob = None
        self.right_prob = None
        self.is_right = None # 1 или 0
        self.ig = None


def entropy_shennon(y: np.array):
    ans = 0
    for i in np.unique(y):
        si = (y == i).sum() / y.shape[0]
        ans -= si * np.log2(si)
    return ans

def IG (*y_arr):
    ans = entropy_shennon(y_arr[0])
    N = y_arr[0].shape[0]
    for yi in y_arr[1:]:
        Ni = yi.shape[0]
        ans -= Ni/N * entropy_shennon(yi) 
    return ans

def get_best_split(X:np.array, y:np.array):
    ig = 0
    split_value = 0
    col_name = ''
    for col in range(X.shape[1]):
        unique_sorted = np.sort(np.unique(X[:, col]))
        separators = np.zeros(unique_sorted.shape[0] - 1)
        for i in range(unique_sorted.shape[0] - 1):
            separators[i] = (unique_sorted[i] + unique_sorted[i+1]) / 2
        
        for sep in separators:
            right = y[X[:, col] > sep]
            left = y[X[:, col] <= sep]
            ig_local = IG(y, right, left)
            
            if ig_local > ig:
                ig = ig_local
                col_name = col
                split_value = sep 
    return (col_name, split_value, ig)

def tree_builder(x, y, curr_depth:int, Node, min_samples_split, max_depth:int = -1, is_right = 0):
        if (x.shape[0] < min_samples_split) or (np.unique(y).shape[0] <=1) or (curr_depth == max_depth):
            if is_right == 1:
                Node.is_right = 1
                Node.right_prob = (y == 1).sum() / y.shape[0]
                Node.left_prob = 0.0
            else:
                Node.is_right = 0
                Node.left_prob = (y == 1).sum() / y.shape[0]
                Node.right_prob = 0.0

        else:
            best_split = get_best_split(x, y)
            
            right_x = x[x[:, best_split[0]] > best_split[1]]
            left_x  = x[x[:,  best_split[0]] <= best_split[1]]
            
            right_y = y[x[:, best_split[0]] > best_split[1]]
            left_y = y[x[:,  best_split[0]] <= best_split[1]]
            
            Node.col = best_split[0]
            Node.sep = best_split[1]
            Node.ig =  best_split[2]
            Node.left =  tree_builder(left_x,  left_y,  curr_depth + 1, TwoPointers(None, None), min_samples_split, max_depth, 0)
            Node.right = tree_builder(right_x, right_y, curr_depth + 1, TwoPointers(None, None), min_samples_split, max_depth, 1)
        return Node
    
def tree_reader(node, indent = ''):
    if (node.left == None) and (node.right == None):
        if node.is_right == 1:
            print(indent, 'right_prob =', node.right_prob)
        else:print(indent, 'left_prob =', node.right_prob)

    else:
        print(indent, 'col:', node.col, 'sep:', node.sep, 'ig:', node.ig)
        tree_reader(node.right, indent + '  ')
        tree_reader(node.left, indent + '  ')
        
class MyTreeClf():

    def __init__(self, max_depth = 5, min_samples_split = 10, max_leafs = None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.leafs_cnt = 0
        self.tree = None

    
    def leaf_counter(self, node):
        if (node.left == None) and (node.right == None):
            self.leafs_cnt +=1

        else:
            self.leaf_counter(node.right)
            self.leaf_counter(node.left )

    def tree_pruner(self, node, min_ig):
        if (node.left.left == None) and (node.left.right == None): # <-- если левая ветка имеет листья
            if (node.ig < min_ig.ig):
                min_ig = node.ig
            
 
    def fit (self, X, y):
        self.tree = TwoPointers(None, None)
        X = X.values
        y = y.values
        tree_builder(X, y, 0, self.tree, self.min_samples_split, max_depth=self.max_depth)
        self.leaf_counter(self.tree)
        
        #if self.leafs_cnt > self.max_leafs:
        #    self.tree_pruner(self.tree)
    
    def display_tree(self):
        tree_reader(self.tree)
    
    def print_leafs_count(self):
        return self.leafs_cnt

In [89]:
df = pd.read_csv('data/Raisin_Dataset.csv')
df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen


In [90]:
df.Class = df.Class.map({'Kecimen':0, 'Besni': 1})

In [96]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns = 'Class'), df.Class, test_size=0.9, random_state=47)

In [97]:
tree = MyTreeClf()

In [100]:
tree.fit(x_train, y_train)

In [99]:
tree.display_tree()

 col: 6 sep: 1126.85 ig: 0.6065890027727684
   col: 0 sep: 95909.5 ig: 0.12719211097303257
     right_prob = 1.0
     col: 0 sep: 90143.5 ig: 0.2951842247970209
       right_prob = 0.25
       col: 1 sep: 415.67553005 ig: 0.1744546479149136
         right_prob = 1.0
         left_prob = 0.0
   col: 3 sep: 0.7590248845 ig: 0.07178377672970987
     right_prob = 0.0
     col: 3 sep: 0.752297 ig: 0.31427482773226467
       right_prob = 1.0
       col: 3 sep: 0.7325572225 ig: 0.12414133222412968
         right_prob = 0.25
         left_prob = 0.0
